# import modules

In [70]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time

# start a selenium driven browser

In [215]:
options = Options()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')

In [231]:
web_url = 'https://www.calorieking.com/us/en/'
# browser= webdriver.Chrome(chrome_options=options).get(web_url)
driver = webdriver.Chrome()
driver.get(web_url)
time.sleep(2)

In [232]:
# get user's key word
result_lst = []
def get_key_word():
    driver.get(web_url)
    time.sleep(2)
    search_terms = input('Please enter a search terms')
    print(f'search for the terms: {search_terms}')
    # search the key words
    search_field = driver.find_element_by_id('keywords')
    search_field.send_keys(search_terms)
    search_btn = driver.find_element_by_xpath('//*[@id="__next"]/main/section[1]/div/form/div/div[2]/button')
    search_btn.click()
    # get search terms results
    brands = driver.find_elements_by_class_name('jss5')
    global result_lst
    result_lst = [ i.find_element_by_class_name('MuiTypography-root').text for i in brands]

get_key_word()

Please enter a search termsapplebee
search for the terms: applebee


In [233]:
target_brand =''
 
def get_brand():
    def check_brand():
        for i in result_lst:
            yield(i)
    brands = check_brand()
    correct = 'F'
    global target_brand
    while correct == 'F':
        try:
            brand_candidate = next(brands)
            correct = input(f'If {brand_candidate} the one you are looking for? T/F: ')
            if correct =='T':
                target_brand = str(brand_candidate)
                pass
            else:
                continue
        except: 
            print('Please try another search tearms.')
            get_key_word()
    
get_brand()

If Applebee's the one you are looking for? T/F: T


In [ ]:
<span class="MuiTypography-root MuiListItemText-primary jss374 MuiTypography-body2 MuiTypography-noWrap MuiTypography-displayBlock">Sirloin Steak (8 oz), without sides</span>

In [243]:
food_lst = []
def get_key_word_checked():
    global target_brand
    driver.get(web_url)
    time.sleep(2)
    meal_terms = input('Please enter the main meal ingrediant.')
    search_terms = target_brand + ' ' + meal_terms
    print(f'search for the terms: {search_terms}')
    # search the key words
    search_field = driver.find_element_by_id('keywords')
    search_field.send_keys(search_terms)
    search_btn = driver.find_element_by_xpath('//*[@id="__next"]/main/section[1]/div/form/div/div[2]/button')
    search_btn.click()
    # get search terms results
    foods = driver.find_elements_by_class_name('jss374')
    global food_lst
    food_lst = [ i.text for i in foods]

get_key_word_checked()

Please enter the main meal ingrediant.steak
search for the terms: Applebee's steak


In [255]:
target_food =''
target_food_url = ''
count = 0
def get_food():
    global count
    global target_food_url
    count=0
    def check_food():
        for i in food_lst:
            yield(i)
    foods = check_food()
    correct = 'F'
    global target_food
    while correct == 'F':
        try:
            food_candidate = next(foods)
            correct = input(f'If {food_candidate} the one you are looking for? T/F: ')
            if correct =='T':
                target_food = str(food_candidate)
                pass
            else:
                count=count+1
                continue
        except: 
            print('Please try another search tearms.')
            get_key_word()
    driver.find_elements_by_class_name('MuiListItem-button')[count].click()
    target_food_url = (driver.current_url)
get_food()

If Sirloin Steak (8 oz), without sides the one you are looking for? T/F: F
If Sirloin Steak (6 oz), without sides the one you are looking for? T/F: T


In [256]:
target_food_url

'https://www.calorieking.com/us/en/foods/f/calories-in-menu-items-sirloin-steak-6-oz-without-sides/Yu-tgIu8RPyayyCmRQxnXw'

# close the browser

In [ ]:
browser.close()